# [Keras Tuner の基礎](https://www.tensorflow.org/tutorials/keras/keras_tuner?hl=ja)

In [1]:
import tensorflow as tf
from tensorflow import keras

import IPython

!pip install -q -U keras-tuner
import kerastuner as kt

# Data Exploration

In [3]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

img_train = img_train.astype('float32') / 255.0
img_test  = img_test.astype('float32') / 255.0

# Modeling and Tuner Configuration

In [13]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    # Keras Tuner Int method
    # parameter: unique_name, min_value, max_value, step
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(10))
        
    # Keras Tuner Choice method
    # parameter: unique_name, values(array)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(optimizer = keras.optimizers.Adam(learning_rate=hp_learning_rate),
                 loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    
    return model


tuner = kt.Hyperband(model_builder, objective='val_accuracy', max_epochs=50, factor=3, directory = 'my_dir', project_name='intro')

class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)
        
tuner.search(img_train, label_train, epochs=10, validation_data=(img_test, label_test), callbacks=[ClearTrainingOutput()])
best_hp = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(best_hp)

Trial 57 Complete [00h 00m 34s]
val_accuracy: 0.8439000248908997

Best val_accuracy So Far: 0.8960999846458435
Total elapsed time: 00h 25m 33s


In [17]:
model = tuner.hypermodel.build(best_hp)
model.fit(img_train, label_train, epochs=10, validation_data=(img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.5867 - accuracy: 0.7931 - val_loss: 0.4331 - val_accuracy: 0.8395
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3688 - accuracy: 0.8635 - val_loss: 0.3736 - val_accuracy: 0.8639
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3301 - accuracy: 0.8793 - val_loss: 0.3615 - val_accuracy: 0.8699
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2992 - accuracy: 0.8878 - val_loss: 0.3570 - val_accuracy: 0.8715
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2752 - accuracy: 0.8978 - val_loss: 0.3443 - val_accuracy: 0.8767
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2575 - accuracy: 0.9041 - val_loss: 0.3411 - val_accuracy: 0.8752
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2528 - accuracy: 0.9055 - val_loss: 0.3454 - val_accuracy: